In [32]:
import os, shutil, glob
from subprocess import Popen, PIPE, STDOUT

import numpy as np
import scipy as sp
import nibabel as nb
from scipy.io import loadmat

# import cortex as cx
import bottleneck as bn
from joblib import Parallel, delayed

# from scipy.ndimage import gaussian_filter, uniform_filter

# from prfpy.rf import *
# from prfpy.timecourse import *
# from prfpy.stimulus import PRFStimulus2D
# from prfpy.model import Iso2DGaussianModel, CSS_Iso2DGaussianModel
# from prfpy.fit import Iso2DGaussianFitter, CSS_Iso2DGaussianFitter

import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
data_path = '/tank/shared/2021/visual/HCP_movie_unprocessed/'

In [21]:
os.chdir(data_path)
md5s = glob.glob('*.zip.md5')
output_bools = []
for md5 in md5s:
    with open(md5, 'r') as f:
        md5_string = f.read()
    cmd = 'md5sum ' + md5.replace('.md5', '')
    p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
    output, stderr = p.communicate()

    print(md5, md5_string.split(' ')[0], str(output)[2:].split(' ')[0], md5_string.split(' ')[0] == str(output)[2:].split(' ')[0])
    output_bools.append([md5, md5_string.split(' ')[0] == str(output)[2:].split(' ')[0]])


360030_7T_tfMRI_MOVIE3_unproc.zip.md5 f64e7e9333eb2b01078c2d1b96dd98bc f64e7e9333eb2b01078c2d1b96dd98bc True
257845_7T_tfMRI_MOVIE2_unproc.zip.md5 7be32dd19c0cea09ce0a8698d87c2e70 7be32dd19c0cea09ce0a8698d87c2e70 True
150423_7T_tfMRI_MOVIE4_unproc.zip.md5 397d076ede8a20210eb35f6dffbe8c97 397d076ede8a20210eb35f6dffbe8c97 True
412528_7T_tfMRI_MOVIE1_unproc.zip.md5 eacc3822a52a099079095ed30f06a8d4 eacc3822a52a099079095ed30f06a8d4 True
617748_7T_tfMRI_MOVIE4_unproc.zip.md5 164198f1c64965bdb2883f599a2dca35 164198f1c64965bdb2883f599a2dca35 True
233326_7T_tfMRI_MOVIE2_unproc.zip.md5 060cabe13b38bdfe319472e9d1c18cc7 060cabe13b38bdfe319472e9d1c18cc7 True
436845_7T_tfMRI_MOVIE3_unproc.zip.md5 d4b70459a1c05c928be128bb0eb6c2ec d4b70459a1c05c928be128bb0eb6c2ec True
180533_7T_tfMRI_MOVIE4_unproc.zip.md5 f50d2d0609087e847defa6a43abe866e f50d2d0609087e847defa6a43abe866e True
187345_7T_tfMRI_MOVIE2_unproc.zip.md5 9b95afc0eee3036282a291dba9834253 9b95afc0eee3036282a291dba9834253 True
971160_7T_tfMRI_MOV

KeyboardInterrupt: 

In [ ]:
def get_ascii_file_from_zip(zipfiles_one_subject, sj):
    for zipfile in zipfiles_one_subject:
        cmd = 'unzip ' + zipfile
        p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
        output, stderr = p.communicate()
        if stderr is not None:
            print(output, stderr)
    
    asc_EL_files = glob.glob(sj + '/unprocessed/7T/tfMRI*/LINKED_DATA/EYETRACKER/*_eyetrack.asc')
    os.makedirs(f'eyetrack/{sj}', exist_ok=True)

    if len(asc_EL_files) > 0:
        for aef in asc_EL_files:
            dest = f'eyetrack/{sj}/{os.path.split(aef)[-1]}'
            print(dest)
            shutil.copyfile(aef, dest)
        
sjs = np.unique(np.array([m.split('_')[0] for m in md5s]))
zip_files_per_sj = [sorted(glob.glob(str(sj)+'*.zip')) for sj in sjs]

Parallel(n_jobs=10, verbose=1)( 
    delayed(get_ascii_file_from_zip)(zipfiles_one_subject=zip_files_per_sj[i], sj=sjs[i]) for i in range(len(sjs)))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
